In [1]:
from mp_api.client import MPRester
import time
import pickle

# Replace with your actual API key
API_KEY = "vhdPJ1STyEi4znoIbrdg6s1j2Q03BQdH"

/Users/pamco116/miniconda3/envs/repo_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Read ICSD entries
with open('full_icsd_entries.txt', 'r') as f:
    icsd_entries = [line.strip() for line in f]



In [3]:
# Build ICSD to MP mapping
with MPRester(API_KEY) as mpr:
    mp_docs = mpr.materials.summary.search(        
        theoretical=False,
        fields=["database_IDs", "material_id"])

Retrieving SummaryDoc documents: 100%|██████████| 49283/49283 [00:09<00:00, 5255.76it/s]


In [4]:

# Build ICSD to MP mapping
with MPRester(API_KEY) as mpr:

    icsd_to_mpid = {}
    for mp_doc in mp_docs:
        mpid = str(mp_doc.material_id)
        for icsd_id in mp_doc.database_IDs.get("icsd", []):
            if icsd_id not in icsd_to_mpid:
                id = icsd_id.replace('icsd-','').zfill(6)
                icsd_to_mpid[id] = []
            icsd_to_mpid[id].append(mpid)

In [12]:
# Find matches
found = []
not_found = []

for icsd in icsd_entries:
    if icsd in icsd_to_mpid:
        found.append((icsd, icsd_to_mpid[icsd]))
    else:
        not_found.append(icsd)

# Save results
with open('mp_found.txt', 'w') as f:
    for icsd, mpids in found:
        f.write(f"{icsd}: {','.join(mpids)}\n")

with open('mp_full_icsd.txt', 'w') as f:
    for icsd, mpids in icsd_to_mpid.items():
        f.write(f"{icsd}: {','.join(mpids)}\n")


print(f"Found in MP: {len(found)}")
print(f"Not found: {len(not_found)}")


Found in MP: 67372
Not found: 68096
